In [32]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import statistics

In [33]:
def drop_weights(G):
    '''Drop the weights from a networkx weighted graph.'''
    for node, edges in nx.to_dict_of_dicts(G).items():
        for edge, attrs in edges.items():
            attrs.pop('weight', None)

In [34]:
def similarita(voti_dep1, voti_dep2):
    uguali = np.sum((voti_dep1 == voti_dep2) & (voti_dep1 != 0) ) # volte in cui hanno votato la stessa  ed erano entrambi presenti

    diversi = np.sum((voti_dep1 != voti_dep2) & (voti_dep1 != 0) & (voti_dep2 != 0) ) # volte in cui hanno votato diversamente ed erano entrambi presenti
    
    if uguali == 0:
        sim = 0
    else:
        sim = uguali/(uguali+diversi)
    return sim

# Graph based on similarity

In [36]:
leg = 'xvii'
year = '2013'

startmonth = 1
endmonth = 13
if (year == '2013' and leg == 'xvii'):
   startmonth = 4
elif (year == '2018' and leg == 'xviii'):
   startmonth = 6
else: 
   startmonth = 1

if (year == '2018' and leg == 'xvii'):
   endmonth = 6
elif (year == '2022' and leg == 'xviii'):
   startmonth = 2 #perché non ci sono dati a gennaio
   endmonth = 9 #perché dopo agosto non ci sono dati
else: 
   endmonth = 13

for mon in range (startmonth, endmonth):
    month = str(mon)
    df = pd.read_csv(f'./{leg}-months/{leg}-{year}_{month}.csv')

   #remove serial absents
   # indici = []
   # absence_tresh = 0.3
   # for i in range(df.shape[0]):
   #    if np.count_nonzero(df.iloc[i,4:]) < absence_tresh*(df.shape[1]-4):
   #       indici.append(i)


   # df.drop(indici, axis = 0, inplace = True)
   # df.shape


    m = np.array(df[df.columns[4:df.shape[1]]])
    m.shape
    A = np.zeros((len(m),len(m))) #inizializzo matrice a 0
    i=0
    j=0
    for i in range(len(m)):
        for j in range(len(m)):
            A[i,j] = similarita(m[i],m[j])
    G = nx.from_numpy_matrix(A)
    G.remove_edges_from(nx.selfloop_edges(G))
    F = G.copy()
    tresh = np.percentile(A, q = 50)
    threshold = tresh
    F.remove_edges_from([(n1, n2) for n1, n2, w in F.edges(data="weight") if w < threshold])
    #drop_weights(F)
    
    print(f'month {mon} --> Nodes: {F.number_of_nodes()} ; Isolated:{len(list(nx.isolates(F)))} ; Non Isolated Nodes: {F.number_of_nodes()-len(list(nx.isolates(F)))}')

    nx.write_weighted_edgelist(F,f'./{leg}-months/edgelist-{leg}-{year}_{month}_weighted.txt')


month 4 --> Nodes: 594 ; Isolated:6 ; Non Isolated Nodes: 588
month 5 --> Nodes: 594 ; Isolated:6 ; Non Isolated Nodes: 588
month 6 --> Nodes: 595 ; Isolated:7 ; Non Isolated Nodes: 588
month 7 --> Nodes: 594 ; Isolated:15 ; Non Isolated Nodes: 579
month 8 --> Nodes: 595 ; Isolated:98 ; Non Isolated Nodes: 497
month 9 --> Nodes: 594 ; Isolated:21 ; Non Isolated Nodes: 573
month 10 --> Nodes: 594 ; Isolated:14 ; Non Isolated Nodes: 580
month 11 --> Nodes: 594 ; Isolated:12 ; Non Isolated Nodes: 582
month 12 --> Nodes: 594 ; Isolated:7 ; Non Isolated Nodes: 587
